## CrewAI核心组件 - Task

+ 基本配置
+ 任务输出
+ 结构化输出

### 基本配置

research_task:
    description: >
        对{topic}进行深入研究
        确保你找到任何有趣且相关的信息，考虑到当前年份是2025年。
    expected output: >
        一个包含10个要点的列表，列出关于{topic}的最相关信息
    agent: researcher

reporting_task:
    description: >
        审查你获得的内容，并将每个主题扩展为报告的完整章节。
        确保报告详细且包含所有相关信息。
    expected output: >
        一份完整的报告，包含主要主题，每个主题都有完整的信息章节。
        以markdown格式输出，不含'```'标记
    agent: reporting_analyst
    output_file: report.md

### 引入代码中执行

In [ ]:
# src/latest_ai_development/crew.py

from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool


@CrewBase
class HelloWorld:
    """HelloWorld crew"""

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config["researcher"],
            verbose=True,
            tools=[SerperDevTool()],
        )

    @agent
    def reporting_analyst(self) -> Agent:
        return Agent(config=self.agents_config["reporting_analyst"], verbose=True)

    @task
    def research_task(self) -> Task:
        return Task(config=self.tasks_config["research_task"])

    @task
    def reporting_task(self) -> Task:
        return Task(config=self.tasks_config["reporting_task"])

    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=[
                self.researcher(),
                self.reporting_analyst(),
            ],
            tasks=[self.research_task(), self.reporting_task()],
            process=Process.sequential,
        )

### 任务输出

In [ ]:
task = Task(
    description="查找并总结最新的人工智能新闻",
    expected_output="最重要的5条人工智能新闻要点总结",
    agent=research_agent,
    tools=[search_tool],
)

# 创建crew实例
crew = Crew(agents=[research_agent], tasks=[task], verbose=True)

# 执行Crew
result = crew.kickoff()

# 获取任务输出
task_output = task.output

# 打印任务输出
# 打印具体的属性
print(f"Task Description: {task_output.description}")
print(f"Task Summary:{task_output.summary}")
print(f"Raw Output: {task_output.raw}")

if task_output.json_dict:
    print(f"Json Output: {json.dumps(task_output.json_dict, indent=2)}")
if task_output.pydantic:
    print("")

### 任务依赖和上下文

In [ ]:
research_task = Task(
    description="研究人工智能领域的最新发展",
    expected_output="人工智能最新发展的列表",
    agent=researcher,
)

analysis_task = Task(
    description="分析研究结果并识别关键趋势",
    expected_output="人工智能趋势分析报告",
    agent=analyst,
    context=[research_task],  # 此任务将等待research task完成后再执行
)

### 结构化输出

In [ ]:
import json

from crewai import Agent, Crew, Process, Task
from pydantic import BaseModel


class Blog(BaseModel):
    title: str
    content: str


blog_agent = Agent(
    role="博客内容生成代理",
    goal="生成博客标题和内容",
    backstory="""你是一位专业内容创作者，擅长制作引人入胜且内容丰富的博客文章。""",
    verbose=Falg,
    allow_delegation=False,
    llm="gpt-4o",
)

task1 = Task(
    description="""创建一个给定主题的博客标题和内容。确保内容不超过200字。""",
    expected_output="一个吸引人的博客标题和编写良好的内容。",
    agent=blog_agent,
    output_pydantic=Blog,
)

# 使用顺序处理过程实例化你的团队
crew = Crew(
    agents=[blog_agent],
    tasks=[task1],
    verbose=True,
    process=Process.sequential,
)

result = crew.kickoff()

# 选项1: 使用字典风格索引访问属性
print("访问属性- 选项1")
title = result["title"]
content = result["content"]
print("标题:", title)
print("内容:", content)

# 选项2: 直接从Pydantic模型访问属性
print("访问属性 - 选项2")
title = result.pydantic.title
content = result.pydantic.content
print("标题:", title)
print("内容:", content)

# 选项3: 使用to_dict()方法访问属性
print("访问属性 - 选项3")
title = output_dict["title"]
content = output_dict["content"]
print("标题:", title)
print("内容:", content)

# 选项4: 打印整个Blog对象p
print("访问属性 - 选项5")
print("博客:", result)